# 완전 연결 신경망

이 노트북을 주피터 노트북 뷰어(nbviewer.jupyter.org)로 보거나 구글 코랩(colab.research.google.com)에서 실행할 수 있습니다.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://nbviewer.jupyter.org/github/rickiepark/nn-in-tf/blob/master/fully_connected.ipynb"><img src="https://jupyter.org/assets/main-logo.svg" width="28" />주피터 노트북 뷰어로 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/nn-in-tf/blob/master/fully_connected.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
</table>

## 보스턴 주택 데이터셋 적재

보스턴 주택 데이터셋을 사용하여 신경망을 훈련하겠습니다. 이 데이터셋은 1970년대 보스턴 인근 지역의 주택에 관한 정보를 기록한 것입니다. 주택에 관한 정보를 바탕으로 주택의 가격을 예측하는 회귀(Regression) 문제입니다.

텐서플로에서 직접 이 데이터를 다운로드할 수 있습니다. 먼저 텐서플로의 케라스 모듈을 임포트합니다.

In [ ]:
from tensorflow import keras

`datasets.boston_housing.load_data()` 함수는 모델 훈련에 사용할 데이터와 테스트에 사용할 데이터를 나누어 넘파이 배열로 전달합니다. 이 데이터를 각각 훈련 세트, 테스트 세트라고 부르겠습니다.

`load_data()` 함수는 기본적으로 전체 데이터에서 20%를 테스트 세트로 떼어내어 반환합니다. y_train과 y_test는 맞추어야 할 주택 가격 데이터입니다. 이를 타깃 데이터라고 부릅니다.

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.boston_housing.load_data()

사용할 데이터의 크기를 확인해 보겠습니다. 넘파이 배열의 shape 속성에서 훈련 세트의 크기를 확인할 수 있습니다. 

In [ ]:
print(x_train.shape, y_train.shape)

`x_train`은 404개의 샘플이 있고 각 샘플은 13개의 특성으로 이루어져 있습니다. 즉 404개의 행이 있고 13개의 열이 있는 2차원 배열입니다. 이를 2차원 행렬로 생각하면 편리합니다.

`y_train`은 404개의 주택 가격을 가진 1차원 배열로 1,000 달러 단위의 값입니다.

`x_test`와 `y_test`의 크기도 확인해 보겠습니다.

In [ ]:
print(x_test.shape, y_test.shape)

특성의 개수는 13개로 동일합니다. 테스트 세트는 102개의 훈련 샘플이 포함되어 있습니다.

## 데이터 전처리

신경망은 입력 특성이 비슷한 스케일을 가져야 잘 작동합니다. 보스턴 주택 데이터의 값의 범위를 상자 수염 그래프로 확인해 보겠습니다.

그래프를 그리기 위해 맷플롯립 패키지를 임포트합니다.

In [ ]:
import matplotlib.pyplot as plt

`boxplot()` 함수로 상자 수염 그래프를 그립니다.

In [ ]:
plt.boxplot(x_train)
plt.show()

그래프에서 볼 수 있듯이 10번째와 12번째 특성은 값의 범위가 매우 큽니다. 이렇게 값의 범위가 크게 다르면 모델이 잘 훈련되기 어렵습니다.

특성 값을 맞추는 대표적인 방법은 표준화입니다. 표준화는 특성별로 평균을 0, 분산을 1로 맞춥니다. 표준화를 하려면 훈련 세트에서 특성별로 평균과 표준 편차를 구해야 합니다.

넘파이 배열 객체의 `mean()`, `std()` 메서드를 사용하면 평균과 표준 편차를 쉽게 구할 수 있습니다. `axis=0`으로 지정하면 특성(열)을 따라 값을 계산합니다.

In [ ]:
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)

x_train_scaled = (x_train - mean) / std

변환한 훈련 세트를 사용하여 모델을 훈련할 것입니다. 훈련된 모델을 테스트 세트로 평가해야 하기 때문에 테스트 세트의 특성도 동일하게 표준화해야 합니다.

여기에서 주의할 점은 테스트 세트의 평균과 표준 편차를 구해서는 안된다는 것입니다. 테스트 세트가 훈련 세트와 다른 비율로 변환된다면 훈련 세트로 훈련한 모델이 쓸모 없게 되기 때문입니다.

꼭 훈련 세트의 값으로 테스트 세트를 변환하세요.

In [ ]:
x_test_scaled = (x_test - mean) / std

## 모델 구성

케라스에서 신경망 모델을 만드는 한가지 방법은 `Sequential` 클래스를 사용하는 것입니다. 이 클래스의 객체를 만든 후 원하는 층을 하나씩 추가할 수 있습니다.

In [ ]:
model = keras.Sequential()

이전 층과 모든 유닛이 연결되는 완전 연결 층을 추가하겠습니다. 텐서플로의 케라스 API에서 완전 연결 층은 `Dense` 클래스에 구현되어 있습니다.

이 클래스의 첫 번째 매개변수는 층에 사용할 유닛 개수입니다. `activation` 매개변수에 활성화 함수를 지정합니다. 자주 사용하는 활성화 함수는 `sigmoid`, `tanh`, `relu` 등입니다. 이 예제에서는 `sigmoid` 활성화 함수를 사용합니다.

`Sequential` 클래스에 추가되는 첫 번째 층에는 `input_shape` 매개변수로 입력의 크기가 지정되어야 합니다. 입력의 크기에서 샘플의 개수(배치)는 훈련시에 정의됩니다. `input_shape` 매개변수에는 배치 차원을 제외한 입력의 크기를 넣습니다. 이 예제에서는 각 샘플이 13개의 특성을 가지므로 `(13,)` 튜플을 전달합니다.

이 예제는 주택 가격을 예측하는 회귀 문제입니다. 즉 마지막 층에 추가된 한 개의 유닛에서 $-\infty$에서 $+\infty$ 사이의 값이 출력됩니다.

In [ ]:
model.add(keras.layers.Dense(64, activation='sigmoid', input_shape=(13,)))
model.add(keras.layers.Dense(1))

모델에 층을 추가한 후 `summary()` 메서드로 전체 모델 구조를 확인할 수 있습니다.

In [ ]:
model.summary()

두 개의 `Dense` 층이 추가되었습니다. 첫 번째 층의 유닛이 64개 이므로 출력의 크기도 64입니다. 역시 배치 차원은 `None`으로 사전에 지정하지 않습니다. 두 번째 층은 출력층으로 하나의 숫자를 출력하기 때문에 출력 크기가 1입니다.

두 개의 완전 연결 층에 있는 가중치의 개수를 확인해 보겠습니다. 첫 번째 층에서 13개의 특성이 64개의 유닛에 모두 곱해지고 유닛마다 하나의 절편이 있습니다.

$13\times64+64=896$

두 번째 층은 이전 층에서 전달된 64개의 입력과 하나의 유닛이 곱해집니다. 이 층에도 유닛마다 하나의 절편이 있습니다.

$64\times1+1=65$

따라서 이 신경망에 있는 총 가중치 개수는 961개 입니다.

## 모델 훈련 및 테스트

모델의 구조를 만든 후 컴파일 단계에서 이 모델이 최소화할 손실 함수를 지정해야 합니다. 또한 최소화에 사용할 최적화 방법인 옵티마이저(optimizer)를 지정해야 합니다.

회귀 문제에 사용하는 대표적인 손실 함수는 평균 제곱 오차(mean square error), 평균 절댓값 오차(mean absolute error)입니다. 여기에서는 평균 절댓값 오차를 사용합니다. 평균 절댓값 오차는 다음 식처럼 모델의 출력과 타깃 사이의 평균 절댓값입니다.

$\text{MAE} = \frac{1}{n}\sum_{i=1}^{n}|a_i - y_i|$

텐서플로는 여러가지 경사 하강법 알고리즘을 제공합니다. 대표적으로 가장 기본적인 `sgd`와 고급 방식인 `rmsprop`, `adam` 등이 있습니다. 이런 알고리즘들은 학습이 진행됨에 따라 가중치 업데이트되는 양을 자동으로 조절합니다. 여기에서는 `adam`을 사용하겠습니다.

In [ ]:
model.compile(loss='mae', optimizer="adam")

모델의 `fit()` 메서드를 사용하여 신경망을 훈련합니다. 이 메서드에는 훈련 세트 `x_train_scaled`와 타깃 `y_train`을 전달합니다. `x_train_scaled`을 사용하여 출력 값을 만들고 이 값이 `y_train`에 가까워지도록 가중치를 조정합니다.

`fit()` 메서드의 `epochs` 매개변수에 반복할 에포크 횟수를 지정합니다. `fit()` 메서의 반환값은 에포크마다 기록한 손실(MAE)이 저장됩니다.

In [ ]:
history = model.fit(x_train_scaled, y_train, epochs=100)

`history` 객체의 `history` 속성에 딕셔너리 형태로 손실 값이 기록되어 있습니다. 맷플롯립으로 그래프를 그려 보겠습니다.

In [ ]:
plt.plot(history.history['loss'])
plt.show()

손실 값이 에포크 초기에 급격히 줄어들고 시간이 지남에 따라 서서시 줄어들고 있습니다.

훈련된 모델이 테스트 세트에서 얼마나 잘 예측하는지 `evaluate()` 메서드를 사용하여 측정해 보죠.

In [ ]:
model.evaluate(x_test_scaled, y_test)

이 모델은 주택 가격을 예측하는데 평균적으로 약 3,300 달러의 오차를 가집니다.

마지막으로 `predict()` 메서드로 테스트 세트에 대한 구체적인 예측 값을 만들어 보겠습니다. 그다음 이 값과 실제 타깃 `y_test`와 비교해 보겠습니다.

In [ ]:
model.predict(x_test_scaled[0:5])

In [ ]:
print(y_test[0:5])